# Installing Whisper

The commands below will install the Python packages needed to use Whisper models and evaluate the transcription results.

In [ ]:
import pandas as pd
import os
import re
from tqdm.auto import tqdm
tqdm.pandas()

In [ ]:
!apt update
!pip install transformers datasets phonemizer
!apt install espeak
!pip install pydub
!pip install transformers --upgrade
!pip install torchaudio
!pip install tqdm --upgrade
!pip install torchaudio --upgrade

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/jonathonf/ffmpeg-4/ubuntu bionic InRelease
0% [Re

In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg

In [ ]:
from transformers import __version__ as transformers_ver
from tqdm import __version__ as tqdm_ver
from torch import __version__ as torch_ver
from torchaudio import __version__ as torchaudio_ver
from pandas import __version__ as pd_ver
print(f"transformers_ver:\t{transformers_ver}")
print(f"tqdm_ver:\t{tqdm_ver}")
print(f"torch_ver:\t{torch_ver}")
print(f"torchaudio_ver:\t{torchaudio_ver}")
print(f"pandas_ver:\t{pd_ver}")

In [ ]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

# model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-xlsr-53-espeak-cv-ft")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-xlsr-53-espeak-cv-ft")

In [ ]:
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-xlsr-53-espeak-cv-ft")

In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)

zip_ref = zipfile.ZipFile("/content/drive/My Drive/tat.zip", 'r')
zip_ref.extractall("/content/asr_tat")
zip_ref.close()

In [ ]:
import pandas as pd

df = pd.read_csv('/content/asr_tat/tat/asr_tat.csv')
df

In [ ]:
df['fpath'] = df['file_name'].map(lambda x: '/content/asr_tat/tat/' + x)
df['fpath']

In [ ]:
df

In [ ]:
import torch
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda install ffmpeg=4.3 -c conda-forge

In [ ]:
import torch
import torchaudio
from tqdm.auto import tqdm
tqdm.pandas()
device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)

def recognizer(fpath):
    try:
        waveform, sample_rate = torchaudio.load(fpath)
        waveform = waveform.to(device)
        logits = model(waveform).logits
        pred_ids = torch.argmax(logits, dim=-1)
        pred_str = processor.batch_decode(pred_ids)[0]
        return pred_str
    except:
        return 0

In [ ]:
df['recognized'] = df['fpath'].progress_apply(recognizer)

In [ ]:
df.head(5)

In [ ]:
!pip install abydos

import abydos

from abydos import distance

phonetic = distance.PhoneticEditDistance()

In [ ]:
phonetic = distance.PhoneticEditDistance()
def phonetic_metric(row):
    try:
        result = phonetic.dist(row['transcription'], row['recognized'])
        return result
    except Exception as e:
        print(e)

In [ ]:
df['phonetic_ev'] = df.progress_apply(phonetic_metric, axis=1)

In [ ]:
df[['transcription', 'recognized', 'phonetic_ev']].sample(10)

In [ ]:
df.phonetic_ev.plot.hist(bins=20)

In [ ]:
df[['transcription', 'recognized']]

In [ ]:
https://github.com/flashlight/flashlight/tree/main/flashlight/app/asr#beam-search-decoder

In [ ]:
uselexicon=true

In [ ]:
df.to_csv('/content/asr_tat/tat/asr_tat_post_model.csv', index=False)

In [ ]:
def recognizer_adv(fpath):
    try:
        waveform, sample_rate = torchaudio.load(fpath)
        waveform = waveform.to(device)
        logits = model(waveform).logits
        pred_ids = torch.argmax(logits, dim=-1)
        pred_str = processor.batch_decode(pred_ids, uselexicon=true)[0]
        return pred_str
    except:
        return 0

In [ ]:
df['recognized_adv'] = df['fpath'].progress_apply(recognizer_adv)

In [ ]:
def phonetic_metric_adv(row):
    try:
        result = phonetic.dist(row['transcription'], row['recognized_adv'])
        return result
    except Exception as e:
        print(e)

In [ ]:
df['phonetic_ev_adv'] = df.progress_apply(phonetic_metric_adv, axis=1)

In [ ]:
df[['transcription', 'recognized_adv', 'phonetic_ev_adv']].sample(10)

In [ ]:
df.phonetic_ev_adv.plot.hist(bins=20)

In [ ]:
df.to_csv('/content/asr_tat/tat/asr_tat_post_model.csv', index=False)